In [ ]:
from utils import *
from Constants import *

In [ ]:
METRIC = 'Root Mean Squared Error-Saved'
PLOT_DIR = f'{SHAPE_MODELS_DIR}/plots_{METRIC}/'
if not os.path.exists(PLOT_DIR):
    os.makedirs(PLOT_DIR)

In [ ]:
temp = np.arange(0, 1, 0.001)

In [ ]:

def get_file_path(model_name, result_type, analysis_suffix=f'_LDS_Analysis_L_64_scaling_True_{METRIC}'):
    model_path = f'{SHAPE_MODELS_DIR}/{model_name}{analysis_suffix}/'
    if result_type == 'SPEC':
        f_path = f'{model_path}/Specificity_results_{model_name}.txt'
    elif result_type == 'GEN':
        f_path = f'{model_path}/generalization_results_{model_name}_K_Fold.txt'
    elif result_type == 'GEN_TIME':
        f_path = f'{model_path}/generalization_missing_time_point_results_K_FOLD_{model_name}.txt'
    return f_path
def plot_box_plots_missing_tp_results(file_1, file_2, label_1, label_2, xlabel, ylabel, plot_name, legend_name='Model Type', val_K = 5):
    plt.style.use('plot_style.txt')
    model_A_rmse = np.loadtxt(file_1)
    model_B_rmse = np.loadtxt(file_2)
    assert model_A_rmse.shape == model_B_rmse.shape
    rmse_cols = np.zeros(2 * model_A_rmse.shape[0] * model_A_rmse.shape[1])
    percent_missing_cols = []
    model_type_cols = []
    P, T_S = model_A_rmse.shape
    start_idx = 0
    missing_time_points = [4, 8, 12, 16, 20]
    val_P = len(missing_time_points)

    for p in range(0, val_P):
        for k in range(val_K):
            p_modified = k * val_P + p
            model_A_rmses_at_t = model_A_rmse[p_modified, :]
            percent_val = f'{str(int(round(missing_time_points[p]/25, 2)*100))}%'
            p_col = [percent_val] * T_S
            model_col = [f'{label_1}'] * T_S
            percent_missing_cols += p_col
            model_type_cols += model_col
            rmse_cols[start_idx:start_idx+T_S] = model_A_rmses_at_t
            start_idx = start_idx + T_S
    assert start_idx == (P * T_S)
    for p in range(0, val_P):
        for k in range(val_K):
            p_modified = k * val_P + p
            model_B_rmses_at_t = model_B_rmse[p_modified, :]
            percent_val = f'{str(int(round(missing_time_points[p]/25, 2)*100))}%'
            p_col = [percent_val] * T_S
            model_col = [f'{label_2}'] * T_S
            percent_missing_cols += p_col
            model_type_cols += model_col
            rmse_cols[start_idx:start_idx+T_S] = model_B_rmses_at_t
            start_idx = start_idx + T_S

    df = pd.DataFrame(list(zip(percent_missing_cols, rmse_cols, model_type_cols)), columns=[f'{xlabel}', f'{ylabel}', f'{legend_name}'])
    df.to_excel('gen_time.xlsx', index=False)
    fig = plt.gcf()
    fig.set_size_inches(17, 6)
    ax = sns.boxplot(x=xlabel, y=ylabel, hue=legend_name, data=df, palette='Set3')
    # ax.set_yticks(np.arange(0, 2, 0.25))
    ax.set_yticks(np.arange(0, 1.5, 0.25))
    ax.set_ylim(ymin=0)
    ax.set_ylim(ymax=1.25)


    ax.set_ylabel(ylabel, fontsize=20)
    ax.set_xlabel(xlabel, fontsize=20)
    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    ax.get_legend().remove()
    # ax2 = sns.swarmplot(x=xlabel, y=ylabel, hue=legend_name, data=df, linewidth=2.5)
    plt.savefig(plot_name)
    



def plot_box_plots(file_1, file_2, label_1, label_2, xlabel, ylabel, plot_name, legend_name='Model Type', reverse=False):
    # For Specificity and Generalization, saved errors
    # Input files saved as (K * TEST_SUB_NUM) X NUM_TIME_POINTS array
    plt.style.use('plot_style.txt')
    model_A_rmse = np.loadtxt(file_1)
    model_B_rmse = np.loadtxt(file_2)
    assert model_A_rmse.shape == model_B_rmse.shape
    rmse_cols = np.zeros(2 * model_A_rmse.shape[0] * model_A_rmse.shape[1])
    time_point_cols = []
    model_type_cols = []
    S, T = model_A_rmse.shape
    # S == K * TEST_SUB_NUM
    if reverse:
        T, S = S, T
    start_idx = 0

    for t in range(0, T):
        model_A_rmses_at_t = model_A_rmse[:, t] # extract all errors for time point = t
        t_col = [t+1] * S # Time point col in df
        model_col = [f'{label_1}'] * S # Label Model Type col for df
        time_point_cols += t_col
        model_type_cols += model_col
        rmse_cols[start_idx:start_idx+S] = model_A_rmses_at_t
        start_idx = start_idx + S
    assert start_idx == (T * S)
    for t in range(0, T):
        model_B_rmses_at_t = model_B_rmse[:, t]
        t_col = [t+1] * S # Time point col in df
        model_col = [f'{label_2}'] * S  # Label Model Type col for df
        time_point_cols += t_col
        model_type_cols += model_col
        rmse_cols[start_idx:start_idx+S] = model_B_rmses_at_t
        start_idx = start_idx + S

    df = pd.DataFrame(list(zip(time_point_cols, rmse_cols, model_type_cols)), columns=[f'{xlabel}', f'{ylabel}', f'{legend_name}'])
    fig = plt.gcf()
    fig.set_size_inches(17, 7)
    print(f' Mean A = {np.mean(rmse_cols[0:(T*S)])}')
    print(f' std dev A = {np.std(rmse_cols[0:(T*S)])}')

    print(f' Mean B = {np.mean(rmse_cols[(T*S):])}')
    print(f' std dev B= {np.std(rmse_cols[(T*S):])}')

    
    ax = sns.boxplot(x=xlabel, y=ylabel, hue=legend_name, data=df, palette="Set3")
    ax.set_ylabel(ylabel, fontsize=20)
    ax.set_xlabel(xlabel, fontsize=20)
    ax.set_yticks(np.arange(0, 1.75, 0.25))
    ax.set_ylim(ymin=0)
    ax.set_ylim(ymax=1.5)
    # ax.set_ylim(ymax=2.0)

    ax.get_legend().remove()

    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    plt.savefig(plot_name)

In [ ]:
result_type = 'SPEC'
model_a_fn = get_file_path(MODEL_A, result_type)
model_b_fn = get_file_path(MODEL_B, result_type)
plot_box_plots(file_1=model_a_fn,
                        file_2=model_b_fn,
                        label_1='Baseline Approach',
                        label_2='Proposed Spatiotemporal Approach',
                        xlabel='Cardiac Cycle Time Point',
                        ylabel='RMSE',
                        legend_name=' ',
                        plot_name=f'{PLOT_DIR}/{result_type}_image.png')

In [ ]:
result_type = 'GEN'
model_a_fn = get_file_path(MODEL_A, result_type)
model_b_fn = get_file_path(MODEL_B, result_type)
plot_box_plots(file_1=model_a_fn,
                        file_2=model_b_fn,
                        label_1='Baseline Approach',
                        label_2='Proposed Spatiotemporal Approach',
                        xlabel='Cardiac Cycle Time Point',
                        ylabel='RMSE',
                        legend_name=' ',
                        plot_name=f'{PLOT_DIR}/{result_type}_image.png')

In [ ]:
result_type = 'GEN_TIME'
model_a_fn = get_file_path(MODEL_A, result_type)
model_b_fn = get_file_path(MODEL_B, result_type)
plot_box_plots_missing_tp_results(file_1=model_a_fn,
                        file_2=model_b_fn,
                        label_1='Baseline Approach',
                        label_2='Proposed Spatiotemporal Approach',
                        xlabel='Percentage of Missing Time Points from Cardiac Cycle',
                        ylabel='RMSE',
                        legend_name=' ',
                        plot_name=f'{PLOT_DIR}/{result_type}.png')

In [ ]:
def plot_box_plots_s2s(file_1, file_2, label_1, label_2, xlabel, ylabel, plot_name, legend_name='Model Type', reverse=False):
    # For Specificity and Generalization, saved errors
    # Input files saved as (K * TEST_SUB_NUM) X NUM_TIME_POINTS array
    plt.style.use('plot_style.txt')
    model_A_rmse = np.loadtxt(file_1)
    
    model_B_rmse = np.loadtxt(file_2)
    assert model_A_rmse.shape == model_B_rmse.shape
    rmse_cols = np.zeros(2 * model_A_rmse.shape[0] * model_A_rmse.shape[1])
    time_point_cols = []
    model_type_cols = []
    T, S = model_A_rmse.shape
    # S == K * TEST_SUB_NUM
    if reverse:
        T, S = S, T
    start_idx = 0

    for t in range(0, T):
        model_A_rmses_at_t = model_A_rmse[t,:] # extract all errors for time point = t
        t_col = [t+1] * S # Time point col in df
        model_col = [f'{label_1}'] * S # Label Model Type col for df
        time_point_cols += t_col
        model_type_cols += model_col
        rmse_cols[start_idx:start_idx+S] = model_A_rmses_at_t
        start_idx = start_idx + S
    assert start_idx == (T * S)
    for t in range(0, T):
        model_B_rmses_at_t = model_B_rmse[t, :]
        t_col = [t+1] * S # Time point col in df
        model_col = [f'{label_2}'] * S  # Label Model Type col for df
        time_point_cols += t_col
        model_type_cols += model_col
        rmse_cols[start_idx:start_idx+S] = model_B_rmses_at_t
        start_idx = start_idx + S

    df = pd.DataFrame(list(zip(time_point_cols, rmse_cols, model_type_cols)), columns=[f'{xlabel}', f'{ylabel}', f'{legend_name}'])
    fig = plt.gcf()
    fig.set_size_inches(15, 7)
    
    ax = sns.boxplot(x=xlabel, y=ylabel, hue=legend_name, data=df, palette="Set3")
    ax.set_ylabel(ylabel, fontsize=20)
    ax.set_xlabel(xlabel, fontsize=20)
    ax.set_yticks(np.arange(0, 5, 0.25)) 
    ax.get_legend().remove()

    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    plt.savefig(plot_name)

In [ ]:
result_type = 'SPEC'
model_a_fn = f'{PROJECT_DIR}/{MODEL_A}_reconst_error_new.txt'

model_b_fn = f'{PROJECT_DIR}/{MODEL_B}_reconst_error_new.txt'

plot_box_plots_s2s(file_1=model_a_fn,
                        file_2=model_b_fn,
                        label_1='Baseline Approach',
                        label_2='Proposed Spatiotemporal Approach',
                        xlabel='Cardiac Cycle Time Point',
                        ylabel='Surface-to-Surface Distance Error',
                        legend_name=' ',
                        plot_name=f'{PLOT_DIR}/S2S_image.png')